In [ ]:
# Deepsparse Engine

from deepsparse import Pipeline
import cv2
import time

# Create YOLO pipeline
yolo_pipeline = Pipeline.create(
    task="yolo",
    model_path="model1.onnx",
    class_names=["vehicle", "numberplate"],
    model_config=None,
)

# Open the video file
vid_cap = cv2.VideoCapture("vid.mp4")

# Initialize variables for FPS calculation
start_time = time.time()
frame_count = 0

while True:
    # Read a frame from the video stream
    success, frame = vid_cap.read()
    if not success:
        break

    # Perform YOLO inference on the current frame
    inference = yolo_pipeline(images=[frame], iou_thres=0.6, conf_thres=0.25)
    boxes = inference.boxes
    scores = inference.scores
    labels = inference.labels
    

    # Draw bounding boxes on the frame
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        x, y, w, h = map(int, box)
        class_label = label
        confidence = score

        # Draw bounding box on the image
        color = (0, 255, 0)  # Green color
        thickness = 2
        cv2.rectangle(frame, (x, y), (w, h), color, thickness)


    # Calculate FPS
    frame_count += 1
    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time

    # Overlay FPS on the frame
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with bounding boxes and FPS
    cv2.imshow('YOLO Output with FPS', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
vid_cap.release()
cv2.destroyAllWindows()


## OpenVino-imgsz-384

In [ ]:
!python detect.py --weights best_openvino_model-150/ --imgsz 360 --nosave --source vid.mp4

In [ ]:
import torch
import time
import numpy as np
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2

target_size = (384, 384)

model = torch.hub.load('ultralytics/yolov5', 'custom', path='best_openvino_model-150/', force_reload=True)

vid_cap = cv2.VideoCapture("vid.mp4")

while True:
    # Read a frame from the video stream
    success, img = vid_cap.read()
    if not success:
        break

    
    
    # Measure the time before processing the frame

    out = Image.fromarray(img)
    resize_transform = transforms.Resize(target_size)
    resized_image = resize_transform(out)
    image_tensor = transforms.ToTensor()(resized_image)
    output = image_tensor.reshape(1,3,384,384)
    start_frame_time = time.time()

    # Perform YOLOv5 inference
    results = model(output)
    
    # Reshape the output to [batch_size, num_anchors, num_attributes] -> [1, 9072, 7]
    output_reshaped = results.view(results.size(0), -1, results.size(-1))
    fps_text = f"FPS: {1 / (time.time() - start_frame_time):.2f}"

    # Extract relevant information
    confidences = output_reshaped[:, :, 4]  # Confidence scores
    class_probs = output_reshaped[:, :, 5:]  # Class probabilities
    coordinates = output_reshaped[:, :, :4]  #Coordinates

    draw = ImageDraw.Draw(out)
    for i in range(coordinates.size(1)):
        box = coordinates[0, i].tolist()
        confidence = confidences[0, i].item()
        if confidence > 0.25:  # Adjust confidence threshold as needed
            x, y, w, h = box
            x=x*5
            w=w*5
            y=y*2.8
            h=h*3
        #x, y, w, h = x * target_size[0], y * target_size[1], w * target_size[0], h * target_size[1]
            draw.rectangle([x - w / 2, y - h / 2, x + w / 2, y + h / 2], outline="red", width=1)
    
    img1 = np.array(out)
    # Display FPS
    
    cv2.putText(img1, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
   
    
    cv2.imshow('YOLO', img1)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture object
vid_cap.release()
    
# Close all OpenCV windows
cv2.destroyAllWindows()

## Openvino-imgsz640

In [ ]:
# runs\\train\\exp4\\weights\\last.pt model
def func():
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='best_openvino_model_int8/', force_reload=True)
    vid_cap = cv2.VideoCapture("vid.mp4")
    vid_cap2 = cv2.VideoCapture("vid.mp4")

    # Initialize variables for measuring FPS
    frame_times = []

    while True:
        # Read a frame from the video stream
        success, img = vid_cap.read()
        success2, img2 = vid_cap2.read()
        if not success:
            break

        img = cv2.resize(img, (540,540))
        img2 = cv2.resize(img2, (540,540))
    
        # Measure the time before processing the frame
        start_frame_time = time.time()

        # Perform YOLOv5 inference
#         print(img.shape)
        results = model(img)
        results2 = model(img2)

        # Display the result
        output_img = np.squeeze(results.render())
        output_img2 = np.squeeze(results2.render())

        # Display FPS
        fps_text = f"FPS: {1 / (time.time() - start_frame_time):.2f}"
        cv2.putText(output_img, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(output_img2, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('YOLO', output_img)
        cv2.imshow('YOLO2', output_img2)

        # Measure the time after processing the frame
        end_frame_time = time.time()
        frame_times.append(end_frame_time - start_frame_time)
        #print(fps_text)
        # Break the loop if 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release the video capture object
    vid_cap.release()
    vid_cap2.release()
    
    # Calculate and print the average FPS
    avg_fps = len(frame_times) / sum(frame_times)
    print(f"Average FPS: {avg_fps:.2f}")

    # Close all OpenCV windows
    cv2.destroyAllWindows()

In [2]:
import time
end_frame_time = time.time()

In [3]:
end_frame_time

1702087417.4244957